In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import timedelta

time_start = time.time()


dataframes = []

#The directory to iterate over
directory = "/Users/gali/Downloads/Tapes"

#Sorted list of file names
sorted_filenames = sorted(os.listdir(directory))

#Iterating over each file in the directory in alphabetical order
for filename in sorted(os.listdir(directory)):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path) and file_path.endswith('.csv'):
        #Reading the CSV file into a DataFrame
        column_names = ['Time', 'Price', 'Volume']
        df = pd.read_csv(file_path,names=column_names)
        #Adding a new column to identify the file source (optional)
        df['source_file'] = filename
        #Appending the DataFrame to the list
        dataframes.append(df)
        

#Concatenating files
combined_df = pd.concat(dataframes, ignore_index=True)
#combined_df = combined_df.drop(columns=['Index'])

#Creating time as a time-delta object
combined_df['Time'] = pd.to_timedelta(combined_df['Time'], unit='s')

#Sorting by source_file and Time
combined_df = combined_df.sort_values(by=['source_file', 'Time'])

print(combined_df.dtypes)
print(combined_df)

combined_df['date'] = combined_df['source_file'].str.extract(r'(\d{4}-\d{2}-\d{2})')
print(combined_df.dtypes)
combined_df['date'] = pd.to_datetime(combined_df['date'])
# 将时间列转换为 timedelta 类型
combined_df['Time'] = pd.to_timedelta(combined_df['Time'])

# 计算日期偏移量并调整时间列
combined_df['Time'] = combined_df['Time'] + combined_df['date'] - combined_df['date'].min()

# 删除辅助列
combined_df.drop(columns=['date'], inplace=True)

# 输出调整后的数据集
print(combined_df)
combined_df.to_csv('tapes.csv',index = False)




# #Calculating the offset for each day
# #First, map each day to a unique number (0 for the first day, 1 for the second, etc.)
# day_mapping = {day: idx for idx, day in enumerate(combined_df['source_file'].unique())}

# #Applying the mapping to create a new column 'day_number'
# combined_df['day_number'] = combined_df['source_file'].map(day_mapping)

# #Calculating the time offset (in seconds) for each day and apply it
# offset_per_day = 8.5 * 60 * 60  #8.5 hours in seconds
# combined_df['adjusted_time'] = combined_df['Time'] + pd.to_timedelta(combined_df['day_number'] * offset_per_day, unit='s')


# #Creating a binary column to indicate the start of a new day, for ARIMAX (unused)
# combined_df['new_day_start'] = combined_df['day_number'].diff().fillna(1).astype(bool).astype(int)

# time_taken = time.time() - time_start
# print("Time taken: " + str(time_taken))
# print(combined_df)

Time           timedelta64[ns]
Price                    int64
Volume                   int64
source_file             object
dtype: object
                          Time  Price  Volume                    source_file
0       0 days 00:00:10.881000    267       1  UoB_Set01_2025-01-02tapes.csv
1       0 days 00:00:11.067000    269       1  UoB_Set01_2025-01-02tapes.csv
2       0 days 00:00:11.222000    267       2  UoB_Set01_2025-01-02tapes.csv
3       0 days 00:00:12.338000    270       2  UoB_Set01_2025-01-02tapes.csv
4       0 days 00:00:13.733000    267       3  UoB_Set01_2025-01-02tapes.csv
...                        ...    ...     ...                            ...
3340496 0 days 08:29:54.706000    108       2  UoB_Set01_2025-07-01tapes.csv
3340497 0 days 08:29:55.481000    108       2  UoB_Set01_2025-07-01tapes.csv
3340498 0 days 08:29:56.349000    108       3  UoB_Set01_2025-07-01tapes.csv
3340499 0 days 08:29:56.597000    108       2  UoB_Set01_2025-07-01tapes.csv
3340500 0 days 

In [17]:
# 将 'Time' 列转换为 datetime 类型
combined_df['Time'] = pd.to_timedelta(combined_df['Time'])

# 按照 'Time' 列排序
combined_df.sort_values(by='Time', inplace=True)

# 提取每天最晚的时间点对应的数据
latest_data = combined_df.groupby(combined_df['Time'].dt.date).tail(1)

# 保留 'Time' 和 'Price' 列
latest_data = latest_data[['Time', 'Price']]

# 删除重复的时间点
latest_data.drop_duplicates(subset='Time', inplace=True)

# 重置索引
latest_data.reset_index(drop=True, inplace=True)

# 输出结果
print(latest_data)

AttributeError: 'TimedeltaProperties' object has no attribute 'date'